In [1]:
%cd ..

/workspace/shai_hulud/ppc_experiments


In [2]:
import argparse
import collections
import lightning
import numpy as np
import pyro
import torch
import logger, train

Seed set to 123


In [3]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [4]:
config, (data, model, trainer) = train.from_file("experiments/ppc_celeba_config.json")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
trainer.fit(model, data, ckpt_path="saved/models/CelebA_Ppc/0512_021258/checkpoint_14.ckpt")

[rank: 0] Seed set to 123
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
[rank: 1] Seed set to 123
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
Missing logger folder: saved/log/CelebA_Ppc/0512_185053/lightning_logs
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

Missing logger folder: saved/log/CelebA_Ppc/0512_185053/lightning_logs


Files already downloaded and verified


[DSI-DGX01:3513597:0:3513597] Caught signal 11 (Segmentation fault: invalid permissions for mapped object at address 0x7f2653958548)
==== backtrace (tid:3513597) ====
 0 0x0000000000042520 __sigaction()  ???:0
 1 0x0000000000016b51 omp_in_final()  ???:0
 2 0x000000000000e05a GOMP_parallel()  ???:0
 3 0x0000000001911e40 at::TensorIteratorBase::for_each()  ???:0
 4 0x000000000670fce3 at::native::(anonymous namespace)::eq_kernel()  BinaryOpsKernel.cpp.AVX2.cpp:0
 5 0x0000000002ce6566 at::(anonymous namespace)::wrapper_CPU_eq_Scalar()  RegisterCPU.cpp:0
 6 0x0000000002ce65f0 c10::impl::wrap_kernel_functor_unboxed_<c10::impl::detail::WrapFunctionIntoFunctor_<c10::CompileTimeFunctionPointer<at::Tensor (at::Tensor const&, c10::Scalar const&), &at::(anonymous namespace)::wrapper_CPU_eq_Scalar>, at::Tensor, c10::guts::typelist::typelist<at::Tensor const&, c10::Scalar const&> >, at::Tensor (at::Tensor const&, c10::Scalar const&)>::call()  RegisterCPU.cpp:0
 7 0x00000000024fbf4e at::_ops::eq_Scal

ProcessExitedException: process 0 terminated with signal SIGSEGV

In [ ]:
model.graph.clear()
model.eval()
model.cpu()
model.particles['valid'].cpu()

In [ ]:
for site in trainer.model.graph.nodes:
    trainer.model.graph.nodes[site]['is_observed'] = trainer.model.graph.nodes[site]['value'] is not None

In [ ]:
xs, _, indices = list(trainer.valid_data_loader)[0]
trainer._load_particles(indices, False)
with pyro.plate_stack("forward", (trainer.num_particles, len(xs))):
    model = pyro.condition(trainer.model, data={k: v['value'] for k, v in trainer.model.graph.nodes.items()})
    x_hats = model().mean(dim=0)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=8, sharex="all", sharey="all", layout="compressed")

for i in range(8):
    orgs = xs[i].squeeze().detach().transpose(0, -1)
    estimates = x_hats[i].squeeze().detach().transpose(0, -1)
    axes[0, i].imshow(orgs)
    axes[1, i].imshow(estimates)

fig.savefig("ppc_celeba_recons.pdf", bbox_inches="tight")
plt.show()

In [ ]:
del xs
del x_hats

In [ ]:
for site in trainer.model.graph.nodes:
    trainer.model.graph.unclamp(site)

In [ ]:
with pyro.plate_stack("forward", (trainer.num_particles, 8)):
    x_hats = trainer.model().mean(dim=0)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=8, sharex="all", sharey="all", layout="compressed")

for i in range(8):
    estimates = x_hats[i].squeeze().detach().transpose(0, -1)
    axes[i].imshow(estimates)

fig.savefig("ppc_celeba_samples.pdf", bbox_inches="tight")
plt.show()

In [ ]:
del x_hats

In [ ]:
SEEDS = [123, 456, 789, 101112, 131415]

In [ ]:
LOG_LIKELIHOODS = torch.zeros(len(SEEDS), requires_grad=False)
MEAN_SQUARED_ERROR = torch.zeros(len(SEEDS), requires_grad=False)

In [ ]:
for (s, SEED) in enumerate(SEEDS):
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    for b, (xs, target, indices) in enumerate(trainer.valid_data_loader):
        trainer._load_particles(indices, False)
    
        with pyro.plate_stack("forward", (trainer.num_particles, len(xs))):
            model = pyro.condition(trainer.model, data={k: v['value'] for k, v in trainer.model.graph.nodes.items()})
            trace = pyro.poutine.trace(model).get_trace()
            x_hats = trace.nodes['X']['value'].mean(dim=0)
        LOG_LIKELIHOODS[s] += trace.nodes['X']['fn'].log_prob(xs).sum()
        MEAN_SQUARED_ERROR[s] += ((xs - x_hats) ** 2).sum(dim=0).mean()

        del xs
        del x_hats
        logger.info("Evaluated likelihood for valid batch %d under seed %s" % (b, s))

    LOG_LIKELIHOODS[s] /= len(trainer.valid_data_loader.sampler)
    MEAN_SQUARED_ERROR[s] /= len(trainer.valid_data_loader.sampler)

In [ ]:
LOG_LIKELIHOODS.mean(), LOG_LIKELIHOODS.std()

In [ ]:
MEAN_SQUARED_ERROR.mean(), MEAN_SQUARED_ERROR.std()